In [1]:
import urllib.request as urllib
from bs4 import BeautifulSoup
import re
from contextlib import closing
import sqlite3
import time
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
options = Options()
options.add_argument('--headless')
 
browser = webdriver.Chrome(chrome_options=options)


c:\users\icech\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


In [44]:
get_each_product_info()

(4.0,
 '以前はシート状のつめみがき&艶出しをたまにしようしていました。この製品は磨き方だけで磨きとツヤ出し両方できるので楽ですし確実に早いです。またしっかりとツヤがでてくれて長期間続きます。男性で少し爪をきれいにしておきたいという人にもいいのではないでしょうか。',
 '簡単にピカピカになります',
 'ビューティー',
 'ネイル',
 'ネイル道具・ケアツール',
 '爪やすり',
 '',
 '',
 '',
 '',
 '',
 '')

In [3]:
low_dbname='1107_low_rate_bakawall_each_prodcuts.db'
high_dbname='1107_high_rate_bakawall_each_prodcuts.db'
Hconn,Hc=get_db_conn(high_dbname)
Lconn,Lc=get_db_conn(low_dbname)



In [4]:
# Hc.execute('''select * from sqlite_master''')
Hc.execute('''create table product_text(id integer primary key,username,baka_star,each_id,pd_url,pd_star,pd_title,pd_text)''')
for i in range(1,11):
    Hc.execute(f'alter table product_text add column category{i}')

Hc.exectute('insert into product_text(username,baka_star,each_id,pd_url) select name,id,rate,url from products')




# Lc.execute('''select * from sqlite_master''')
Lc.execute('''create table product_text(id integer primary key,username,baka_star,each_id,pd_url,pd_star,pd_title,pd_text)''')
for i in range(1,11):
    Lc.execute(f'alter table product_text add column category{i}')

Lc.exectute('insert into product_text(username,baka_star,each_id,pd_url) select name,id,rate,url from products')

for h in Hc:
    print(h)

OperationalError: database is locked

In [12]:
#idとrateが逆になっている
# Hc.execute('alter table products add column category')
# Hc.execute('alter table products add column product_name')


# Hc.execute('''select distinct * from product_text ''')
Hc.execute('''select * from sqlite_master''')
    


('table', 'products', 'products', 2, 'CREATE TABLE products (id,name,rate,url,review,text, category, product_name)')
